Scrape healthgrades for a list of medical oncologists in NYC

In [9]:
import requests, json, time
base_api = 'https://www.healthgrades.com/api3/usearch'
params = {
    'userLocalTime': '13:35',
    'practicing_specialties': 'ps456',
    'what': 'Oncology',
    'entityCode': 'PS592',
    'searchType': 'PracticingSpecialty',
    'spec': 67,
    'category': 'provider',
    'where': 'New York, NY',
    'pt': '40.71455, -74.007118',
    'sort.provider': 'bestmatch',
    # sessionId: S3c20f1ef717e279c
    # requestId: Rdbfc23779ea047aa
    'pageNum': 1,
    'isFirstRequest': 'true',
    'debug': 'false',
    'isAtlas': 'false'
}
itemsPerPage = 36
resp = requests.get(base_api, params=params)
resp = json.loads(resp.text)
numPages = int(resp['search']['searchResults']['totalCount'] / itemsPerPage) + 1
doctornames = []
doctoraddresses = []
providerdetails = []
for page in range(1, numPages + 1):
    params['pageNum'] = page
    resp = requests.get(base_api, params=params)
    resp = json.loads(resp.text)
    time.sleep(1)
    providers = resp['search']['searchResults']['provider']
    for provider in providers['results']:
        doctornames.append(provider['displayName'])
        doctoraddresses.append(provider['address'])
        providerdetails.append(provider)
        

check if data makes sense

In [31]:
providerdetails[0]

{'acceptsNewPatients': False,
 'address': {'line1': '160 E 34th St Fl 4', 'line2': 'New York, NY 10016'},
 'addresses': [{'address1': '160 E 34th St Fl 4',
   'officeCode': 'OYXLBSP',
   'officeGuid': 'CBD7151B-344D-47ED-8AB4-EDBE8F8C1BA7',
   'location': {'cityName': 'New York',
    'cityAndState': 'New York, NY',
    'encodedCityAndState': 'New+York%2C+NY',
    'cityLink': 'new-york',
    'stateLink': 'ny-new-york',
    'cityStateZipBestMatch': 'New York, NY 10016',
    'providerCount': 0,
    'displayCityName': 'New York, NY',
    'isStateOnly': False,
    'isTopCity': False,
    'region': {'regionAbbreviation': 'NY',
     'regionName': 'New York',
     'isCountry': False,
     'regionDisplay': 'New York (NY)',
     'zip': '10016',
     'regionLink': 'ny-new-york',
     'regionReverseLink': 'new-york-ny',
     'coordinates': {'latitude': 40.74572,
      'longitude': -73.97882,
      'latLon': '40.74572,-73.97882'},
     'nation': 'USA',
     'hasProviders': False,
     'supressRegio

In [30]:
from pandas.io.json import json_normalize
import pandas as pd
df = pd.io.json.json_normalize(providerdetails)

convert to pandas dataframe and save

In [28]:
list(df)

['aboutMe',
 'acceptsNewPatients',
 'address.line1',
 'address.line2',
 'addresses',
 'affiliatedHospitalIDs',
 'age',
 'availability.hasAny',
 'availability.nextDayAppointments',
 'availability.sameDayAppointments',
 'carePhilosophy.fullText',
 'carePhilosophy.isShortened',
 'carePhilosophy.text',
 'displayLastName',
 'displayName',
 'displayOffice.address',
 'displayOffice.cd',
 'displayOffice.city',
 'displayOffice.displayOfficeCityState',
 'displayOffice.distanceInMiles',
 'displayOffice.id',
 'displayOffice.isPrimaryOffice',
 'displayOffice.lat',
 'displayOffice.lng',
 'displayOffice.name',
 'displayOffice.officeName',
 'displayOffice.phoneNumbers',
 'displayOffice.rank',
 'displayOffice.state',
 'displayOffice.type',
 'displayOffice.url',
 'displayOffice.zip',
 'distance',
 'enhancedMessage.message',
 'enhancedMessage.phoneNumber',
 'enhancedMessage.selectedFacility',
 'enhancedMessage.selectedName',
 'enhancedMessage.selectedSponsor',
 'gender',
 'hasNextDayAppointment',
 'hasOa

save as a .pkl file

In [29]:
df.to_pickle('Healthgradesdoctorlist.pkl')